# Analýza výstavby dopravní infrastruktury v zemích Evropské Unie
### Semestrální práce – UZPR (Skupina B)
### Členové:
- František Gurecký
- Veronika Ciencialová

---

## Motivace
Tato práce se zaměřuje na **výstavbu dálniční a železniční infrastruktury** v zemích EU a její vývoj v čase.

Zvláštní důraz je kladen na **Českou republiku**, která je porovnávána s ostatními členskými státy EU.

---

## Cíle práce
Cílem této analýzy je:

- **zjistit, jak si Česká republika vede** ve výstavbě dálnic a železnic ve srovnání s ostatními státy EU  
- **porovnat jednotlivé státy Evropské unie** z hlediska rozvoje dopravní infrastruktury  
- **analyzovat průměrné roční přírůstky** dopravní infrastruktury  
- **sledovat vývoj celkové délky infrastruktury v čase**

---

## Použitá data
Databáze **Eurostat**, která poskytuje oficiální a srovnatelná statistická data pro státy Evropské unie.

Dále datová sada **DATA250** poskytovaná ČÚZK.

# Postup zpracování
## Odkazy na data
Prvním krokem bude stažení potřebných dat. Níže jsou API odkazy na stažení dat.

In [ ]:
railway_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/rail_if_tracks$defaultview/1.0?compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'
highway_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/road_if_motorwa$defaultview/1.0?compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'
population_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/tps00001/1.0/*.*.*?c[freq]=A&c[indic_de]=JAN&c[geo]=EU27_2020,EA20,EA19,BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,FX,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,IS,LI,NO,CH,UK,BA,ME,MD,MK,GE,AL,RS,TR,UA,XK,AD,BY,MC,RU,SM,AM,AZ&c[TIME_PERIOD]=2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025&compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'

## Načítání dat z Eurostatu

Data jsou získávána přímo z veřejného API databáze **Eurostat** ve formátu CSV.  
Pro zajištění přehlednosti a opakovatelnosti kódu je vytvořena pomocná funkce,
která stáhne data z daného URL odkazu, ověří úspěšnost požadavku a připraví data k dalšímu zpracování v knihovně **pandas**.


In [ ]:
import requests
import io

def download_file(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Failed to retrieve data, code: {response.status_code}')
        return None

    file = io.BytesIO(response.content)

    return file

## Stažení datových sad

V této části jsou stažena všechna potřebná data z databáze Eurostat:

- **železniční infrastruktura** – celková délka železničních tratí  
- **dálniční infrastruktura** – celková délka dálnic
- **populační data** – využitá pro přepočty hodnot na obyvatele  

Všechna data jsou stahována jednotným způsobem pomocí dříve definované funkce.

In [ ]:
railway_file = download_file(railway_url)
highway_file = download_file(highway_url)
population_file = download_file(population_url)

## Načtení dat do prostředí pandas

Stažené CSV soubory jsou nyní načteny do knihovny **pandas**, která umožňuje
efektivní práci s tabulkovými daty.

Každý datový zdroj je uložen do samostatného *DataFrame*.


In [ ]:
import pandas as pd

railway_data = pd.read_csv(railway_file)
highway_data = pd.read_csv(highway_file)
population_data = pd.read_csv(population_file)

## Struktura datových sad

Všechny použité datové sady pochází z databáze **Eurostat** a mají velmi podobnou
tabulkovou strukturu, což usnadňuje jejich další zpracování a vzájemné porovnávání.

Každý řádek představuje **jedno pozorování pro konkrétní stát a rok**.

---

### Společné atributy dat
Napříč všemi datovými sadami se objevují následující klíčové sloupce:

- **geo** – kód státu a jeho název v angličtině
- **TIME_PERIOD** – rok, ke kterému se hodnota vztahuje  
- **OBS_VALUE** – sledovaný ukazatel:
  - délka železnic v km,
  - délka dálnic v km,
  - počet obyvatel  
- **freq** – časová frekvence dat (roční)  
- **unit** – jednotka ukazatele (kilometry / počet osob)


In [ ]:
railway_data.head()

In [ ]:
highway_data.head()

In [ ]:
population_data.head()

## Výběr států a vynechání souhrnných řádků

V datových sadách se kromě jednotlivých států nachází také **souhrnné řádky** pro:
- Eurozónu (`EA19`, `EA20`)  
- Evropskou unii (`EU27_2020`)  

Tyto souhrnné hodnoty **nebudeme v analýze používat**, protože chceme porovnávat **konkrétní státy** a hodnoty ČR vůči ostatním zemím.

---

### Speciální případy

- `XK:Kosovo*` obsahuje hvězdičku, protože **Eurostat uvádí Kosovo s omezeným uznáním** a s poznámkou o jeho statutárním postavení.  
- Ostatní státy jsou standardně kódovány dvoupísmenným kódem ISO (např. `CZ` pro Českou republiku) a názvem v angličtině.


In [ ]:
population_data['geo'].unique()

## Odstranění souhrnných řádků

V předchozím kroku jsme si všimli, že některé hodnoty obsahují souhrnné údaje pro
Eurozónu a EU. Tyto řádky nejsou relevantní pro porovnávání jednotlivých států, proto nyní provedeme **filtrování**:

- Vyřadíme všechny řádky, jejichž kód státu (`geo`) obsahuje závorky `(` nebo `)`, 
  protože tyto značí souhrnné hodnoty nebo poznámky.

In [ ]:
highway_data = highway_data[~highway_data['geo'].str.contains(r'[()]', regex=True)]
railway_data = railway_data[~railway_data['geo'].str.contains(r'[()]', regex=True)]
population_data = population_data[~population_data['geo'].str.contains(r'[()]', regex=True)]

## Ověření filtrování souhrnných řádků

Po odstranění souhrnných hodnot pro Eurozónu a EU zkontrolujeme,
že zůstaly pouze jednotlivé státy.  


In [ ]:
population_data['geo'].unique()

## Rozdělení sloupce `geo` na kód a název státu

Sloupec `geo` obsahuje kombinaci **kódu státu** a **názvu státu** ve tvaru `CZ:Czechia`.  
Pro snadnější práci a čitelnější vizualizace jej nyní rozdělíme do dvou samostatných sloupců:

- **CNTR_CODE** – dvoupísmenný kód státu (např. `CZ`)  
- **COUNTRY_NAME** – plný název státu (např. `Czechia`)  

Označení je také konzistentní s prostorovými daty použitými dále.

In [ ]:
def split_geo_column(df, source_col='geo'):
    parts = df[source_col].str.split(':', n=1, expand=True)
    df['CNTR_CODE'] = parts.iloc[:, 0]
    df['COUNTRY_NAME'] = parts.iloc[:, 1]
    return df

In [ ]:
railway_data = split_geo_column(railway_data)
highway_data = split_geo_column(highway_data)
population_data = split_geo_column(population_data)

## Ověření rozdělení `geo`

Po rozdělení opět zkontrolujeme,
že se operace provedla správně.  


In [ ]:
population_data.head()

## Prostorová data (NUTS)

Pro vizualizace na mapě je použita **geometrie území států a regionů EU** z formátu
GeoPackage (`.gpkg`). 

- Soubor `NUTS_RG_10M_2024_3035.gpkg` obsahuje geometrické tvary regionů NUTS 2024.  
- Tyto informace umožní vizualizovat dopravní infrastrukturu přímo na mapě a propojit ji
  s daty o železnicích, dálnicích a populaci.  

> *Z důvodu problémů s načítáním z odkazu byla data načtena lokálně.*


In [ ]:
import geopandas as gpd
nuts_file = "NUTS_RG_10M_2024_3035.gpkg"
nuts_data = gpd.read_file(nuts_file)

## Přehled prostorových dat (NUTS)

Po načtení souboru `NUTS_RG_10M_2024_3035.gpkg` získáváme informace o **geometrii regionů EU**:

- Každý řádek představuje **jeden region NUTS**.  
- Klíčové sloupce obvykle zahrnují:
  - `NUTS_ID` – unikátní kód regionu,
  - `LEVL_CODE` – kód úrovně NUTS,
  - `CNTR_CODE` – dvoupísmenný kód státu (shodný s našimi tabulkovými daty),
  - `NAME_LATN` – název regionu,
  - `geometry` – polygon představující hranice regionu.

In [ ]:
nuts_data.head()

## Výběr geometrie států

Pro naši analýzu potřebujeme pouze **úroveň států (Level 0)**.

- Sloupec `LEVL_CODE` určuje úroveň regionu v hierarchii NUTS:  
  - `0` odpovídá státům
  - `1`, `2`, `3` jsou dále pro jednotlivé podregiony  
- Vybereme pouze řádky s `LEVL_CODE == 0`, abychom získali **geometrii celých států EU**, kterou lze propojit s tabulkovými daty.


In [ ]:
countries = nuts_data[nuts_data["LEVL_CODE"] == 0]

## Výběr společného časového období

Datasety železnic, dálnic a populace mají **různou dostupnost dat** pro jednotlivé roky.  
Pro lepší srovnání provedeme:

1. **Zjištění dostupných let** v každém datasetu.  
2. **Průnik těchto let** – ponecháme pouze roky, které jsou dostupné ve všech třech datasetech.  
3. **Filtrace dat** tak, aby všechny datasety obsahovaly stejné roky.  

Tím zajistíme, že následné výpočty průměrů, růstu a vizualizace budou **konzistentní**.


In [ ]:
def available_years(df):
    return set(
        df.loc[df['OBS_VALUE'].notna(), 'TIME_PERIOD'].unique()
    )

def drop_out_of_range(df, min_year, max_year, year_col='TIME_PERIOD'):
    return df[
        df[year_col].between(min_year, max_year)
    ]

In [ ]:
common_years = (
    available_years(railway_data)
    & available_years(highway_data)
    & available_years(population_data)
)

min_year = min(common_years)
max_year = max(common_years)

railway_data = drop_out_of_range(railway_data, min_year, max_year)
highway_data = drop_out_of_range(highway_data, min_year, max_year)
population_data = drop_out_of_range(population_data, min_year, max_year)

print(railway_data["TIME_PERIOD"].unique())
print(highway_data["TIME_PERIOD"].unique())
print(population_data["TIME_PERIOD"].unique())

## Příprava dat pro sloučení a tvorbu časových řad

Pro vizualizace a analýzu chceme každý stát zastoupen **jedním řádkem** a každý rok bude mít **samostatný sloupec** pro:

- délku železnic (`railway_{rok}`)  
- délku dálnic (`highway_{rok}`)  
- počet obyvatel (`population_{rok}`)  

---

### Postup:

1. Iterujeme přes všechny **společné roky**.  
2. Pro každý rok:
   - Vybereme hodnoty z jednotlivých datasetů.  
   - Přejmenujeme sloupec `OBS_VALUE` na `railway_{rok}`, `highway_{rok}`, `population_{rok}`.  
3. Propojení s geopandas `countries` přes `CNTR_CODE` metodou `merge`.  

Tímto způsobem získáme **časovou řadu** vhodnou pro další výpočty a vizualizace.


In [ ]:
for y in common_years:
    railway_y = railway_data[railway_data['TIME_PERIOD'] == y].groupby('CNTR_CODE')['OBS_VALUE'].first().reset_index().rename(columns={'OBS_VALUE': f'railway_{y}'})
    highway_y = highway_data[highway_data['TIME_PERIOD'] == y].groupby('CNTR_CODE')['OBS_VALUE'].first().reset_index().rename(columns={'OBS_VALUE': f'highway_{y}'})
    population_y = population_data[population_data['TIME_PERIOD'] == y].groupby('CNTR_CODE')['OBS_VALUE'].first().reset_index().rename(columns={'OBS_VALUE': f'population_{y}'})
    
    countries = countries.merge(railway_y, on='CNTR_CODE', how='left')
    countries = countries.merge(highway_y, on='CNTR_CODE', how='left')
    countries = countries.merge(population_y, on='CNTR_CODE', how='left')

countries.head()

## Interaktivní mapa pro vybraný rok

Pomocí funkce `explore_column` můžeme rychle vizualizovat požadované sloupce na mapě států EU.  

Funkce:

1. Vytvoří kopii GeoDataFrame s geometrií a názvy států.  
2. Přidá vybrané sloupce pro vizualizaci.  
3. Vrátí interaktivní mapu s **barevným zobrazením hodnot prvního sloupce**.  


In [ ]:
def explore_column(df, columns, legend=True, colors='OrRd'):
    df_copy = df[['NAME_LATN', 'geometry']].copy()
    for column in columns:
        df_copy[column] = df[column]

    # We excpect the first column to be the one to display legend for
    return df_copy.explore(columns[0], legend=legend, cmap=colors)

In [ ]:
explore_column(countries, ['railway_2015', 'population_2015'])

## Normalizace dat podle populace a rozlohy

Pro spravedlivé srovnání států je důležité zohlednit **velikost populace a rozlohu**:

- **PC – per capita**: délka železnic nebo dálnic na milion obyvatel (`railway_pc`, `highway_pc`).  
- **PA – per area**: délka železnic nebo dálnic na 1 km² území státu (`railway_pa`, `highway_pa`).

Tato data mají již vyšší vypovídací hodnotu než samotná délka.


In [ ]:
for y in common_years:
    countries[f"railway_pc_{y}"] = countries[f"railway_{y}"] / (countries[f"population_{y}"] / 1e6)   # PC - per capita
    countries[f"railway_pa_{y}"] = countries[f"railway_{y}"] / (countries.area / 1e6)         # PA - per area
    
    countries[f"highway_pc_{y}"] = countries[f"highway_{y}"] / (countries[f"population_{y}"] / 1e6)   # PC - per capita
    countries[f"highway_pa_{y}"] = countries[f"highway_{y}"] / (countries.area / 1e6)         # PA - per area

In [ ]:
explore_column(countries, ['highway_pc_2022', 'population_2022'])

In [ ]:
explore_column(countries, ['highway_pa_2022', 'population_2022'])

## Výpočet meziročního nárůstu (Year-over-Year)

Pro každý stát a rok počítáme **rozdíl oproti předchozímu roku**:

- **railway_pc_yoy** – meziroční změna délky železnic na milion obyvatel  
- **railway_pa_yoy** – meziroční změna délky železnic na km²  
- **highway_pc_yoy** – meziroční změna délky dálnic na milion obyvatel  
- **highway_pa_yoy** – meziroční změna délky dálnic na km²  

Můžeme pak lépe posoudit kde infrastruktura **rostla nejrychleji**.


In [ ]:
for year in range(min_year+1, max_year+1):
    current = year
    previous = current - 1
    
    countries[f'railway_pc_yoy_{current}'] = countries[f'railway_pc_{current}'] - countries[f'railway_pc_{previous}']
    countries[f'railway_pa_yoy_{current}'] = countries[f'railway_pa_{current}'] - countries[f'railway_pa_{previous}']
    countries[f'highway_pc_yoy_{current}'] = countries[f'highway_pc_{current}'] - countries[f'highway_pc_{previous}']
    countries[f'highway_pa_yoy_{current}'] = countries[f'highway_pa_{current}'] - countries[f'highway_pa_{previous}']

## Průměrné žebříčkové umístění států podle meziročního nárůstu infrastruktury

V této části počítáme **průměrné umístění států na žebříčku**, založené na ročním nárůstu infrastruktury:

1. Pro každý rok, kromě prvního, se **státy seřadí podle meziročního přírůstku**:
   - železnice na obyvatele (`railway_pc_yoy`)  
   - železnice na km² (`railway_pa_yoy`)  
   - dálnice na obyvatele (`highway_pc_yoy`)  
   - dálnice na km² (`highway_pa_yoy`)  


2. Rank je přiřazen s metodou `min`, aby byly **rovnoměrně hodnoceny státy se stejným přírůstkem**.  

3. Hodnocení se **sčítá přes všechny roky a následně průměruje**, čímž vzniká **jednotný index intenzity výstavby infrastruktury**.  

4. Státy bez dat mají automaticky přiřazeno poslední místo v žebříčku.

Tyto indexy mohou definitivně říct, kterému státu se daří **nejlépe stavět infrastrukturu**.


In [ ]:
# Init
countries['rank_rw_pc'] = 0
countries['rank_rw_pa'] = 0
countries['rank_hw_pc'] = 0
countries['rank_hw_pa'] = 0

year_count = 0
country_count = len(countries.index)

for year in common_years:
    if year == min_year:
        continue
    year_count += 1
    countries['rank_rw_pc'] += countries[f'railway_pc_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)
    countries['rank_rw_pa'] += countries[f'railway_pa_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)
    countries['rank_hw_pc'] += countries[f'highway_pc_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)
    countries['rank_hw_pa'] += countries[f'highway_pa_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)

# Calculate average ranking across all years
countries['rank_rw_pc'] /= year_count
countries['rank_rw_pa'] /= year_count
countries['rank_hw_pc'] /= year_count
countries['rank_hw_pa'] /= year_count

# Replace 0s with NaN for countries with no data
countries['rank_rw_pc'] = countries['rank_rw_pc'].replace(0, float('nan'))
countries['rank_rw_pa'] = countries['rank_rw_pa'].replace(0, float('nan'))
countries['rank_hw_pc'] = countries['rank_hw_pc'].replace(0, float('nan'))
countries['rank_hw_pa'] = countries['rank_hw_pa'].replace(0, float('nan'))

In [ ]:
explore_column(countries, ['rank_rw_pc'])

## Top 10 států podle průměrného ročního růstu infrastruktury na obyvatele nebo plochu

Po výpočtu **průměrného ranku meziročního nárůstu infrastruktury** můžeme:

1. Seřadit státy podle průměrného umístění (`rank_rw_pc`) – menší hodnota = vyšší růst.  
2. Zobrazit **top 10 států**, které v průběhu let **budovaly infrastrukturu nejintenzivněji vzhledem k počtu obyvatel nebo ploše**.  
3. Porovnat tabulku s interaktivní mapou (`explore`), aby bylo možné **vidět prostorové rozložení intenzity výstavby**.


In [ ]:
countries[['CNTR_CODE', 'NAME_LATN', 'rank_hw_pa']].sort_values('rank_hw_pa').head(10)

# Detailní analýza infrastruktury na území ČR

Načteme **geometrická data českého území** z datasetu Data250 (ČÚZK).

Funkce `download_and_extract_shapefiles` stáhne data z odkazu v *zip* složce, extrahuje tato data do dočasné složky *tmp** v adresáři projektu a vrátí cestu k této složce.

K názvu složky bylo nutné také dodat jméno podsložky, která se v datech bohužel nachází.

In [ ]:
import zipfile

def download_and_extract_shapefiles(url, extract_to='tmp'):
    zip_file = download_file(url)
    
    # Extract the zip file
    with zipfile.ZipFile(zip_file) as z:
        z.extractall(extract_to)
        # Print extracted *.shp files
        print("Extracted files:")
        for file in z.namelist():
            if file.endswith('.shp'):
                print(f"  - {file}")
    
    return extract_to

# Download and extract
data250_url = 'https://openzu.cuzk.gov.cz/opendata/Data250/epsg-5514/data250.zip'
data250_path = download_and_extract_shapefiles(data250_url) + '/DATA250_2025_SHP'

## Použitá data

Námi analyzované dálnice a železnice se nachází v souborech:

- `RailrdL.shp` = železnice (linie)
- `RoadL.shp` = silnice (linie)
- `BuiltupA.shp` = zástavba (polygony)
- `PolbndRegDA.shp` = kraje (polygony)

Tato data tedy načteme pomocí **geopandas**

In [ ]:
railrdL = gpd.read_file(f'{data250_path}/RailrdL.shp')
roadL = gpd.read_file(f'{data250_path}/RoadL.shp')
builtupA = gpd.read_file(f'{data250_path}/BuiltupA.shp')
regions = gpd.read_file(f'{data250_path}/PolbndRegDA.shp')

## Souřadnicový systém

Informace o použitém zobrazení pro data nalezneme v atributu `crs`

Jelikož jsou data ze stejného zdroje, není nutno data transformovat pomocí funkce `to_crs()`

In [ ]:
regions.crs

## Vizualizace železnic, silnic a zastavěných území ČR

Vytvořili jsme **kombinovanou mapu českého území**, která zobrazuje:

- **Železnice** (`railrdL`) – tmavě zelená  
- **Silnice/dálnice** (`roadL`) – tmavě modrá  
- **Zastavěné oblasti** (`builtupA`) – tmavě červená  

Jedná se pouze o **rychlý prostorový přehled**, kde se nachází hlavní infrastruktura a jak je rozložena vzhledem k zastavěným oblastem.


In [ ]:
plot = railrdL.plot(color='green', alpha=0.5)
roadL.plot(ax=plot, color='blue', alpha=0.5)
builtupA.plot(ax=plot, color='red', alpha=0.5)

# Kolik procent zástavby je od dálnic vzdáleno méně než x km vzdušnou čarou.
---
## Filtrace dálnic z vrstev silnic
Nejprve je nutné vůbec zjistit, kde v datech nalezneme dálnice.

Zobrazme si tedy hlavičku a zkusme odhadnout, o jaký atribut se jedná.

In [ ]:
roadL.head()

### Dokumentace napoví

Podrobnosti o významu jednotlivých atributů nalezneme zde: https://geoportal.cuzk.cz/(S(tpll15t4qqkcecepehbxtig0))/Dokumenty/Data250_katalog.pdf

Atribut `NA3` obsahuje klasifikaci typu komunikace má dálnice kód **'16'**.

Provedeme tedy filtraci a zobrazíme dálnice v grafu.

In [ ]:
hwL = roadL[roadL['NA3'] == '16']
plot = hwL.plot(color='darkgreen', alpha=0.5)

## Kolik zastavěných ploch leží blízko dálnic?

Nyní, když už víme, kde dálnice v našich datech hledat, můžeme zjistit, **jak dostupné jsou dálnice ze zástaveb**.

Postup jsme provedli následujícím způsobem:

1. Vytvořili jsme **buffer kolem dálnic**, abychom získali okolí dálnic.  
2. Zjistili jsme, které **zastavěné plochy se s tímto bufferem překrývají**.  
3. Spočítali jsme **plochu těchto zástaveb v blízkosti** a vyjádřili ji jako procento z celkové plochy zástavby.  
4. Výsledek jsme vizualizovali:  
   - **modře**: buffer kolem dálnic  
   - **zeleně**: všechny zastavěné plochy  
   - **červeně**: zastavěné plochy uvnitř bufferu


In [ ]:
import matplotlib.pyplot as plt

distance = 10e3

hwL_buffered = hwL.copy()
hwL_buffered['geometry'] = hwL_buffered.buffer(distance)

hw_union = hwL_buffered.geometry.union_all()

colides = builtupA.copy()
colides['intersected'] = colides.geometry.intersection(hw_union)
colides['intersected_area'] = colides['intersected'].area

total_builtup_area = builtupA.geometry.area.sum()
intersected_area = colides['intersected_area'].sum()
percentage = (intersected_area / total_builtup_area) * 100

fig, ax = plt.subplots(figsize=(12, 10))

gpd.GeoSeries([hw_union]).plot(ax=ax, color='blue', alpha=0.3)

builtupA.plot(ax=ax, color='darkgreen', alpha=0.5)

colides_with_intersection = colides[colides['intersected_area'] > 0]
colides_with_intersection.set_geometry('intersected').plot(ax=ax, color='darkred', alpha=0.7)

ax.set_title(f'Zastavěné plochy do {distance/1000}km od dálnic: \n{percentage:.2f}%')
plt.show()

## Které kraje mají nejhustší infrastrukturu?

V této části se zaměřujeme na **dopravní hustotu silnic a železnic v jednotlivých regionech**.

Nejdříve jsme zjistili, jak silnice a železnice **překrývají hranice krajů** a spočítali jejich délky.  
Poté jsme **sečetli délky všech komunikací a tratí v každém kraji** a vztáhli je k ploše kraje, čímž vznikla **hustota infrastruktury** vyjádřená v km na km².  

Díky tomu můžeme snadno porovnat kraje a zjistit, **kde je infrastruktura nejintenzivnější** a jak je regionálně rozložena.  


In [ ]:
roads_by_region = gpd.overlay(
    roadL, regions, how="intersection", keep_geom_type=True
)
railways_by_region = gpd.overlay(
    railrdL, regions, how="intersection", keep_geom_type=True
)

roads_by_region["road_length_km"] = roads_by_region.geometry.length / 1e3
railways_by_region["railway_length_km"] = railways_by_region.geometry.length / 1e3

roads_by_region.geometry

Dále tedy sečteme **délky silnic a železnic**, spočítáme **plochu** a vypočteme **hustotu**.

In [ ]:
road_density = (
    roads_by_region
    .groupby("NAMN", as_index=False)["road_length_km"]
    .sum()
)

railway_density = (
    railways_by_region
    .groupby("NAMN", as_index=False)["railway_length_km"]
    .sum()
)

regions["area_km2"] = regions.geometry.area / 1e6

regions_dense = regions.merge(road_density, on="NAMN", how="left")
regions_dense = regions_dense.merge(railway_density, on="NAMN", how="left")

regions_dense["road_density_km_per_km2"] = (
    regions_dense["road_length_km"] / regions_dense["area_km2"]
)

regions_dense["railway_density_km_per_km2"] = (
    regions_dense["railway_length_km"] / regions_dense["area_km2"]
)

**Odpověď zobrazíme v mapě**

In [ ]:
df_copy = regions_dense[['NAMN', 'geometry', 'railway_density_km_per_km2', 'road_density_km_per_km2']].copy()

df_copy.explore('road_density_km_per_km2', legend=True, cmap='OrRd')

# Závěr a shrnutí

V této semestrální práci jsme se zaměřili na **analýzu výstavby dopravní infrastruktury v zemích Evropské unie**, se zvláštním důrazem na **Českou republiku**.

České republice se daří poměrně dobře, ve výstavbě dálnic na plochu se dokonce umístila na první příčce.

---

### Shrnutí použitých znalostí a postupů

Během práce jsme využili:

- **Knihovny pandas a geopandas** pro zpracování a agregaci dat  
- **Prostorovou analýzu a vizualizace** s geopandas a matplotlib  
- **Průniky a buffery** pro analýzu dostupnosti infrastruktury ze zástavby  
- **Normalizaci dat** podle populace a rozlohy  

Díky kombinaci **datové analýzy, statistiky a prostorové analýzy** jsme byli schopni posoudit **intenzitu výstavby infrastruktury**, její **prostorové rozložení** a **srovnání mezi státy a regiony**.